In [4]:
import os
import pandas as pd
from PIL import Image

from imblearn.over_sampling import SMOTE

import torch
from torch.utils.data import Dataset

from torchvision.transforms import ToTensor, ToPILImage

from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
cwd=os.path.dirname(os.getcwd())

def make_images(meta,img_dir,train):
    images=[]
    labels=[]
    if train:
        for idx in range(len(meta)):
            folder_path=os.path.join(img_dir, meta.path[idx])
            for img in os.listdir(folder_path):
                if '._' in img:
                    continue
                images.append(os.path.join(folder_path,img))
                labels.append((('incorrect' in img)+('normal' in img)*2)*6+(meta.gender[idx]=='female')*3+(30<=meta.age[idx])+(60<=meta.age[idx]))
    else:
        for img_id in meta.ImageID:
            images.append(os.path.join(img_dir, img_id))
    return images,labels

class ImageDataset(Dataset):
    def __init__(self,train=True):
        self.train=train
        self.md=['info','train']
        self.path=[os.path.join(cwd,'input/data/eval'),os.path.join(cwd,'input/data/train')]
        self.meta=pd.read_csv(os.path.join(self.path[train], f'{self.md[train]}.csv'))
        self.img_dir=os.path.join(self.path[train],'images')
        self.classes=[('Wear','Incorrect','Not Wear'),('남','여'),('<30','>=30 and <60','>=60')]
        
        self.images,self.labels=make_images(self.meta,self.img_dir,train)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image=ToTensor()(Image.open(self.images[idx]))
        if self.train:
            label=torch.tensor(self.labels[idx])
        else:
            label=0
        return image,label

data=ImageDataset()
print(len(data))
with tqdm(enumerate(data)) as pbar:
    for n,(image,label) in pbar:
        if n==0:
            images=image.view(-1)
            continue
        images=torch.vstack((images,image.view(-1)))
        if n==279:
            print(image.shape)
            break
        
labels=torch.tensor(data.labels[:280])

print(images.shape,labels.shape)
print(pd.Series(labels).value_counts())

smote=SMOTE(random_state=2021)
smote_images,smote_labels=smote.fit_resample(images,labels)

print(smote_images.shape,smote_labels.shape)
print(pd.Series(smote_labels).value_counts())

for n,(img,lab) in enumerate(zip(smote_images,smote_labels)):
    print(img.shape)
    img=torch.tensor(img).view(3, 512, 384)
    (ToPILImage()(img)).save(os.path.join(cwd,f'input/data/SMOTE/{n}_{lab}.jpeg'))

18900


torch.Size([3, 512, 384])

torch.Size([280, 589824]) torch.Size([280])
4     165
1      35
16     33
10     33
13      7
7       7
dtype: int64
(990, 589824) (990,)
16    165
13    165
10    165
7     165
4     165
1     165
dtype: int64
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(589824,)
(5

In [1]:
print(images.shape,labels.shape)

NameError: name 'images' is not defined

In [ ]:
!nvidia-smi